安装依赖  
pip install -q openai python-dotenv  
pip install openai-agents

In [49]:
#读取.env
import os
from dotenv import load_dotenv
from agents import Agent, trace, Runner, function_tool, set_default_openai_client, set_default_openai_api
import json
import requests
import base64
import re
from typing import Union
from collections import OrderedDict
from openai import AsyncOpenAI
from IPython.display import Markdown, display
from openai.types.chat import ChatCompletionMessageParam
from typing import List, Dict, Any

load_dotenv(override=True)

True

In [50]:
#检查api
flowus_api_key = os.getenv('FLOWUS_API_KEY')
imgbb_api_key = os.getenv('IMGBB_API_KEY')

if flowus_api_key:
    print(f"FLOWUS API key 存在且开头为{flowus_api_key[:8]}")
else:
    print("FLOWUS API key 尚未设置")

if imgbb_api_key:
    print(f"IMGBB API key 存在且开头为{imgbb_api_key[:8]}")
else:
    print("IMGBB API key 尚未设置")

FLOWUS API key 存在且开头为sk-HNrhY
IMGBB API key 存在且开头为d1c4555f


In [51]:
# 初始化client
flowus_client = AsyncOpenAI(
    api_key=flowus_api_key,
    base_url="https://api.xty.app/v1"
)

# 设置默认 client 和默认 API
set_default_openai_client(flowus_client)
set_default_openai_api("chat_completions")

In [74]:
#上传image以供Agent识别
def upload_image_to_imgbb(image_path):
    with open(image_path, "rb") as f:
        encoded_image = base64.b64encode(f.read())

    response = requests.post(
        "https://api.imgbb.com/1/upload",
        data={
            "key": imgbb_api_key,
            "image": encoded_image
        }
    )

    if response.status_code == 200:
        return response.json()["data"]["url"]
    else:
        raise Exception(f"❌ 上传失败: {response.text}")
def build_image_records(folder_path):
    image_records = []
    for fname in os.listdir(folder_path):
        if fname.lower().endswith((".png", ".jpg", ".jpeg", ".webp")):
            fpath = os.path.join(folder_path, fname)
            try:
                url = upload_image_to_imgbb(fpath)
                image_records.append({
                    "filename": fname,
                    "image_url": url
                })
                print(f"✅ 上传成功: {fname}")
            except Exception as e:
                print(f"❌ 上传失败: {fname} - {e}")
    return image_records
def load_frida_log(folder_path: str) -> str:
    """
    加载 Frida 日志全文内容。
    """
    try:
        for filename in os.listdir(folder_path):
            if filename.lower().endswith(".log"):
                log_path = os.path.join(folder_path, filename)
                try:
                    with open(log_path, "r", encoding="utf-8") as f:
                        content = f.read()
                except UnicodeDecodeError:
                    with open(log_path, "r", encoding="gbk", errors="ignore") as f:
                        content = f.read()
                print(f"✅ 成功读取 Frida 日志: {filename}，长度 {len(content)} 字符")
                return content
        print(f"❌ 未找到 .log 文件: {folder_path}")
        return ""
    except Exception as e:
        print(f"❌ 读取日志失败: {e}")
        return ""

In [ ]:
@function_tool
async def analyze_image_url_with_flowus(filename: str, image_url: str) -> dict:
    print(f"正在识别图片: {filename}")
    messages = [
        {
            "role": "system",
            "content": "你是一个 OCR 图像识别专家，请识别图片中的所有文字，并判断其中是否包含权限说明。"
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": (
                        f"请识别图像文件 {filename} 中的所有可见文字内容，并完成以下任务：\n\n"
                        "1. 提取所有文字，原样输出为 raw_text。\n"
                        "2. 判断是否存在权限说明，如有，请提取该段并输出为 permission_info。\n"
                        "注意以下判断标准：\n"
                        "- 属于权限说明的关键词：获取权限、申请权限、收集信息、获取位置信息、访问相册、使用该权限、实现功能、为您推荐、验证 IMEI 等。\n"
                        "- 不属于权限说明的内容包括：允许/拒绝、Allow/Deny、系统弹窗提示、是否允许 XXX 访问某权限。\n"
                        "只提取文字，不做解释，不输出格式说明。\n"
                        "直接返回两个字段：raw_text 和 permission_info。"
                    )
                },
                {"type": "image_url", "image_url": {"url": image_url}}
            ]
        }
    ]
    try:
        response = await flowus_client.chat.completions.create(
            model="gemini-2.5-flash",
            messages=messages,
            max_tokens=2048
        )
        content = response.choices[0].message.content.strip()
    except Exception as e:
        print(f"\n❌ 模型调用失败 for {filename}：{e}\n")
        return {
            "filename": filename,
            "timestamp": "error",
            "raw_text": "",
            "permission_info": None
        }

    match = re.search(r'(\d{13})', filename)
    timestamp = match.group(1) if match else "null"
    raw_text = ""
    permission_info = None
    raw_match = re.search(r'raw_text\s*[:：]\s*(.+)', content, re.DOTALL)
    perm_match = re.search(r'permission_info\s*[:：]\s*(.+)', content, re.DOTALL)
    if raw_match:
        raw_text = raw_match.group(1).strip()
    if perm_match:
        permission_info = perm_match.group(1).strip()
        if permission_info.lower() in {"null", "none", "无"}:
            permission_info = None
    return {
        "filename": filename,
        "timestamp": timestamp,
        "raw_text": raw_text,
        "permission_info": permission_info
    }
@function_tool
async def filter_related_apis_from_fridalog(fridalog: str, types: list[str]) -> dict:
    """
    使用大模型分析 Frida 日志文本，自动识别与指定权限类型相关的 API 调用段落及上下文，无需正则匹配。

    参数:
        fridalog: 完整的 Frida hook 日志字符串
        types: 权限类型列表（如 ["location", "audio", "calendar"]）

    返回:
        dict: {"location": [...], "audio": [...]}
    """
    prompt = [
        {
            "role": "system",
            "content": (
                "你是一名 Android 安全分析专家，正在分析 Frida hook 日志。\n"
                "日志包含多段以 nBacktrace 开头的调用栈段，每段可能含有异常头、调用栈、时间戳等。\n"
                "你需要：\n"
                "1. 按照用户指定的权限类型进行分类；\n"
                "2. 每段调用栈整体视为一个字符串（包括 nBacktrace、异常、at...等），放入字段 backtrace；\n"
                "3. 尝试提取该段的时间戳（如有）作为 timestamp（13 位或 ISO 格式均可）；\n"
                "4. 注意：只输出你看到的 `权限类型包括：[...]` 中明确列出的类型。其他类型一律不要包含，哪怕为空数组也不要出现。\n\n"
                "最终返回标准 JSON，格式如下：\n"
                "{\n"
                "  \"location\": [\n"
                "    { \"backtrace\": \"完整调用段\", \"timestamp\": \"时间戳\" },\n"
                "    ...\n"
                "  ],\n"
                "  \"audio\": [ ... ]\n"
                "}"
            )
        },
        {
            "role": "user",
            "content": (
                f"权限类型包括：{types}\n\n"
                f"以下是完整 Frida hook 日志：\n{fridalog}"
            )
        }
    ]
    response = await flowus_client.chat.completions.create(
        model="gpt-4o",
        messages=prompt,
        temperature=0.2,
        max_tokens=4096
    )
    result_text = response.choices[0].message.content.strip()
    return result_text

In [82]:
# 四个Agent的prompt
instruction1 = """
你是一个截图分析专家，用户会给你多张截图的图片 URL 和对应文件名。你必须逐张调用工具 analyze_image_url_with_flowus 来对每张图片进行识别分析，并根据返回结果完成以下任务：

请注意，你只输出 JSON，不加任何描述性前缀或结尾。不要输出“以下是分析结果”之类的话。

1. 对每张截图使用工具 analyze_image_url_with_flowus，获取以下字段：
   - `filename`: 原始图片文件名
   - `raw_text`: 图像中识别出的所有文字内容
   - `permission_info`: 若存在权限说明，则为提取的权限说明文本，否则为 null
   - `timestamp`: 从文件名中提取的 13 位时间戳（如 abc_1753201142089.png → 1753201142089）

2. 根据每张图片的 permission_info 字段判断：
   - 若 permission_info 非空，表示该图片包含权限说明，agent2.success 为 true；
   - 若 permission_info 为 null，表示该图片未识别到权限说明，agent2.success 为 false；
   - 请将 permission_info 原文填入 agent2.permission_info。

3. 对每条成功识别出权限说明的图片，还需判断其用途类型（type），归类为以下之一：
["storage", "location", "take picture and record video", "photo media and files", "audio", "剪切板", "悬浮窗", "manage phone call", "查看使用的应用", "身体活动数据", "contact", "calendar"]

若 permission_info 非空，根据其内容判断 type（如下方映射表）；
若 permission_info 为 null，但 raw_text 中出现了“Allow XX to …”或“是否允许 XX 使用 … 权限”等系统权限弹窗提示，也应从中推断 type 并视为成功（agent3.success 为 true）；
若无法推断用途，则 agent3.success 为 false，type 和 timestamp 也为 null。

请根据 permission_info 的内容判断用途类型：
   - “文件、下载、缓存” → storage
   - “定位、地图、位置” → location
   - “拍照、摄像、录像” → take picture and record video
   - “相册、图库、媒体文件” → photo media and files
   - “麦克风、语音、录音” → audio
   - “剪贴板” → 剪切板
   - “悬浮窗、画中画” → 悬浮窗
   - “拨打电话、通话记录” → manage phone call
   - “使用情况访问权限” → 查看使用的应用
   - “步数、加速度传感器” → 身体活动数据
   - “联系人、通讯录” → contact
   - “日历、日程、提醒” → calendar

若未识别出权限说明，则 type 为 null。

4. 所有 raw_text 字段应按图片逐项记录到 debug_output 中。

5. 最终请输出如下格式 JSON：

如果至少有一张截图识别出权限说明（即 agent2 中存在 success 为 true 的项）：
json
{
  "from": "agent1",
  "status": true,
  "agent2": [
    {
      "filename": "mapapp_1753200000000.png",
      "success": true,
      "permission_info": "我们会使用您的位置信息推荐附近服务"
    },
    {
      "filename": "callapp_1755787770000.jpg",
      "success": false,
      "permission_info": null
    }
  ],
  "agent3": [
    {
      "filename": "mapapp_1753200000000.png",
      "success": true,
      "timestamp": "1753200000000",
      "type": "location"
    },
    {
      "filename": "callapp_1755787770000.jpg",
      "success": true,
      "timestamp": "1755787770000",
      "type": "manage phone call"
    }
  ],
  "debug_output": [
    {
      "filename": "mapapp_1753200000000.png",
      "raw_text": "我们会使用您的位置信息推荐附近服务"
    },
    {
      "filename": "callapp_1755787770000.jpg",
      "raw_text": "Allow 全民吉他 to make and manage phone calls? Allow Deny 全民吉他APP www.qmjita.cn"
    }
  ]
}

如果所有截图都未识别到权限说明（即 agent2 全部为 false，无论是否存在系统权限请求提示）,如果没有识别到权限说明，但 raw_text 中出现了类似 “Allow XX to …” 的系统权限请求提示，也必须生成对应的 agent3 条目；否则 agent3 保持为空。：
json
{
  "from": "agent1",
  "status": false,
  "agent2": [],
  "agent3": [
    {
      "filename": "xxx.png",
      "success": true,
      "timestamp": "1755787770000",
      "type": "manage phone call"
    }
  ],
  "debug_output": [
    {
      "filename": "xxx.png",
      "raw_text": "Allow 全民吉他 to make and manage phone calls? Allow Deny 全民吉他APP www.qmjita.cn"
    }
  ]
}

请注意：

必须调用工具 analyze_image_url_with_flowus 分析每一张截图，不能自己识别图像内容；

所有输出必须严格符合预设的JSON 格式，不添加解释、不包含额外字段。

你应对所有输入图片循环调用工具并汇总处理结果，最后统一构造 JSON 返回。
"""
instruction2 = """
你是一个表述规范性分析专家，用户会提供多个权限说明文本及其对应截图文件名。你需要从以下三个方面判断每条说明是否符合表述规范性：
【明确具体】
说明应明确指出申请该权限的具体功能或使用场景，而不是宽泛或空洞的说法。描述中应体现“使用目的 + 权限类型 + 具体操作”，避免仅说明“需要”或“为了更好使用”等含糊理由。
✅ 合规示例：
“访问相册用于上传头像”
“位置权限用于显示附近商家”
❌ 违规示例：
“需要相册权限以优化体验”
“为了更好地推荐服务，需要位置信息”
“基于 LBS 提供 POI 信息”
【简洁易懂】
描述应语言直白，通俗易懂，不要使用专业术语（如 LBS、SDK、POI）、不常见缩写，或让用户难以理解的技术表述。避免堆叠名词、长句或复杂结构。
✅ 合规示例：
“位置权限用于推荐附近店铺”
“相机权限用于扫描二维码”
❌ 违规示例：
“启用地理位置服务（LBS）以支持 POI 展示”
“本权限用于启用增强现实（AR）模块”
【无诱导性】
描述不得带有暗示必须授权、制造授权压力、或将授权与正常使用绑定的表达。避免使用“必须”“否则无法使用”“强烈建议开启”等具有引导性或恐吓性的语句。
✅ 合规示例：
“通讯录权限用于联系人同步”
“麦克风权限用于语音输入功能”
❌ 违规示例：
“拒绝授权将影响核心功能”
“开启权限可解锁更多服务”
“必须开启权限才能正常使用 App”
如权限说明存在以下任何问题，即应判定为不合规（success: false）并给出简明理由：
描述不具体：未交代用途或场景
表达不清晰：含技术术语或难懂语言
具有诱导性：强迫或暗示必须授权
如果三项标准均符合，则为合规（success: true），reason 填 null。

你的任务是：对每条权限说明进行判断，并给出是否符合规范性（true 或 false），假如不符合表述规范性则生成一句话中文理由说明（不超过 50 个字，写入 `reason` 字段中），以帮助用户理解判断依据。

你将收到一个 JSON，其中包含字段 "package"，请务必将这个字段原样保留在你的输出 JSON 中，不要更改为图片名或其他内容。注意：你不得生成或更改 `package` 字段的内容，只能复用输入 JSON 中的原始值。

最终输出格式如下，必须为合法 JSON：

如果所有图片说明都符合规范性：
json
{
  "from": "agent2",
  "package": "xxx",
  "status": true,
  "agent4": [
    {
      "filename": "xxx.png",
      "success": true,
      "permission_info": "xxx",
      "reason": null
    },
    {
      "filename": "yyy.png",
      "success": true,
      "permission_info": "xxx",
      "reason": null
    }
  ]
}
如果有图片的说明不符合规范性：
json
{
  "from": "agent2",
  "package": "xxx",
  "status": false,
  "agent4": [
    {
      "filename": "xxx.png",
      "success": false,
      "permission_info": "xxx",
      "reason": "描述模糊且带有诱导性"
    },
    {
      "filename": "yyy.png",
      "success": true,
      "permission_info": "xxx",
      "reason": null
    }
  ]
}
请注意：

无论是否合规，输出中都必须包含字段：from ,status ,agent4,agent4字段中必须包含字段filename, success, permission_info, reason
其中 reason 必须为字符串或 null，长度不超过 50 字，注意：请从输入数据中提取 "package" 字段，并原样保留在输出 JSON 中。
输出必须是严格的 JSON 格式，不得添加任何解释或注释。你必须确保输出中的所有字符串字段都是合法的 JSON 字符串：
对所有字符串值（尤其是 permission_info 和 reason）中的 `"`（双引号）必须使用反斜杠转义为 `\"`。
禁止输出非法 JSON，例如中文引号（如“”）包裹英文词语、“请求"定位"权限”这类不转义的内容。
注意：如果你输出的 JSON 无法被 `json.loads()` 成功解析，将被判定为失败。请务必在输出前确保 JSON 格式正确。
"""
instruction3 = """
你是一名 Android 安全分析专家，正在协助分析某移动应用的敏感权限调用行为是否存在风险。
你会收到如下 JSON 输入：
{
  "package": "com.example.app",
  "types": ["location", "audio"],
  "description": "与权限相关的文字说明",
  "fridalog_path": "/path/to/frida.log"
}
你的任务包括以下几步：

1. 使用工具函数 `filter_related_apis_from_fridalog(fridalog_text, types)`，从指定路径的 Frida 日志中提取与权限类型相关的调用段；
2. 工具输出为 dict 格式，如 {"location": [ {"backtrace": "...", "timestamp": "..."}, ... ]}；
3. 在输出结果中筛选 `timestamp` 字段匹配 Agent1 中提供的时间戳；
4. 整理所有命中的调用段，按类型拼接为一段完整调用日志内容（joined_log）；
5. 审阅 joined_log 中的所有敏感 API 调用，结合 `description` 字段披露的信息，判断是否存在用途不一致或未披露行为；
5. 将方法、用途、调用段落整合为以下格式：
json
{
  "api": "android.location.LocationManager.requestLocationUpdates",
  "usage": "导航",
  "backtrace": "nBacktrace:\nat android.location.LocationManager.requestLocationUpdates(LocationManager.java:123)\nat ...",
  "reason": "50字左右的简要说明，说明你判断为用途场景的理由"
}
【最终输出格式要求】

你必须仅输出一个统一的 JSON 结果，用于表示所有段落合并后的合规性分析结论：
若检测到用途不一致的敏感 API，输出：
json
{
  "from": "agent3",
  "package": "com.example.app",
  "status": "false",
  "matched_apis": [
    {
      "api": "android.location.LocationManager.requestLocationUpdates",
      "usage": "导航",
      "backtrace": "nBacktrace:\nat android.location.LocationManager.requestLocationUpdates(LocationManager.java:123)\nat ...",
      "reason": "50字左右的简要说明，说明你判断为用途场景的理由"
    },
    {
      "api": "android.media.AudioRecord.startRecording",
      "usage": "录音",
      "backtrace": "nBacktrace:\nat android.media.AudioRecord.startRecording(AudioRecord.java:88)\nat ...",
      "reason": "50字左右的简要说明，说明你判断为用途场景的理由"
    }
  ]
}
若未检测到任何用途不一致的调用，输出：
json
{
  "from": "agent3",
  "package": "com.example.app",
  "status": "true",
  "matched_apis": []
}
【注意事项】
只允许输出一个合法 JSON 对象，不允许输出 Markdown、注释、说明、分析或文字解释；严禁输出多段 JSON 或嵌入说明。
method 中每项为包含 api、usage、backtrace、reason 的字典；
backtrace 必须是对应敏感 API 所属调用段，从 nBacktrace: 开始，直到最后一个 at 语句为止；
package 必须与输入保持一致；status 为字符串类型，值为 "true" 或 "false"；matched_apis 必须是数组，即使为空也必须存在；不输出多余的字段
"""
instruction4 = """
你是一名熟悉 GDPR、PIPL 等移动隐私法规的专家，任务是根据自动分析模块输出的结构化数据，为某移动应用生成一份隐私合规性评估与优化报告。

你将收到两个 JSON 输入对象，分别来自前置分析 Agent 的输出，包含字段 "from" 标记其来源（如 "agent1"、"agent2"、"agent3"），请根据来源类型执行以下规则：

【情形 A】：输入包含 "agent1" 和 "agent3" 的输出时，按以下逻辑处理：

从 "agent1" 中读取字段 "status"：

若为 true，表示该应用的权限说明可见，可继续进行后续分析；

若为 false，表示该应用的权限说明缺失或不可见，此时应在报告中强调该问题的严重性，但仍需继续分析 Agent3 的输出，用于评估其实际敏感权限行为是否合理合规。

从 "agent3" 中读取字段 "status"：

若为 true，说明在 Frida 动态分析日志中未发现敏感 API 与截图行为存在强关联，判定为信息真实性无明显问题；

若为 false，说明存在敏感 API 调用与截图时间戳匹配的行为，但该调用用途在说明中未被披露，存在用途未披露的合规风险。此时应提取：

"package" 字段：代表当前测试应用的包名；

"matched_api" 字段：表示与截图行为高度相关的敏感 API 调用。

报告中应结合检测到的 matched_api，指出该应用存在敏感权限用途未明确披露的情况，属于高风险项，并建议根据法规条款进行披露补充或权限调用优化。

【情形 B】输入包含 "agent2" 和 "agent3" 的输出时，按以下逻辑处理：

遍历 "agent2" 中的 "agent4" 字段，提取所有 "success": false 的项，收集字段：
   - filename
   - permission_info
   - reason
对于每条 success: false 的权限说明条目，请结合原始 permission_info 和对应的 reason 字段（如“描述模糊”、“存在诱导性”等），生成一条优化后的权限说明文本，优化说明应严格遵循以下要求：
【明确具体】：权限用途应当清晰、具体，避免使用泛泛或模糊的表达，使用户能够理解该权限的实际用途。
   - 合规示例：
     - “访问相册用于上传头像”
     - “位置权限用于显示附近商家”
   - 违规示例：
     - “需要相册权限以优化体验”
     - “基于 LBS 提供 POI 信息”
【简洁易懂】：描述应使用通俗语言，避免使用专业术语、复杂表达或晦涩语言。
   - 合规示例：
     - “位置权限用于显示附近商家”
     - “相机权限用于扫描二维码”
   - 违规示例：
     - “基于地理位置服务（LBS）提供 POI 信息”
     - “本权限用于启用应用增强现实（AR）功能”
【无诱导性】：不得包含误导性或诱导用户授权的表述，例如强制授权、解锁功能等暗示。
   - 合规示例：
     - “通讯录权限用于联系人同步功能”
     - “麦克风权限用于语音输入”
   - 违规示例：
     - “允许通讯录权限可解锁高级服务”
     - “拒绝麦克风权限可能影响核心功能”

然后从 "agent3" 中读取字段 "status"：

若为 true，说明在 Frida 动态分析日志中未发现敏感 API 与截图行为存在强关联，判定为信息真实性无明显问题；

若为 false，说明存在敏感 API 调用与截图时间戳匹配的行为，但该调用用途在说明中未被披露，存在用途未披露的合规风险。此时应提取：

"package" 字段：代表当前测试应用的包名；

"matched_api" 字段：表示与截图行为高度相关的敏感 API 调用。

报告中应结合检测到的 matched_api，指出该应用存在敏感权限用途未明确披露的情况，属于高风险项，并建议根据法规条款进行披露补充或权限调用优化。

---

【报告结构建议】你必须输出一份结构完整、逻辑清晰、语言专业的隐私合规性报告，包括以下部分（结构可根据输入内容动态调整）：

1. 权限说明概览
   - 是否提供权限说明；
   - 权限类型与初步描述。

2. 表述规范性分析
   - 明确性、可读性、无诱导性逐项评估；
   - 指出存在问题的句式；
   - 提出重写建议。

3. 用途一致性分析
   - 说明与实际 API 调用的比对结果；
   - 涉及 SDK 或模块；
   - 若推理存在不确定性，请说明依据不足。

4. 合规性评估与改进建议
   - 是否违反 GDPR/PIPL 要求；
   - 提供具体条款引用（如 [GDPR-13.1]）；
   - 输出清晰、改写后的权限说明建议文本。

【注意】：
- 所有引用法规请使用 `[GDPR-xx]` 或 `[PIPL-xx]` 等形式；
- 输出为完整报告文本，不得添加 JSON 格式内容，可以使用Markdown格式输出；
- 不要简单罗列输入数据，而是整合分析并生成连贯的分析性文字；
- 所有条目应以中文输出。
"""

In [83]:
#生成四个Agent实例
agent1 = Agent(
    name="权限说明识别处理Agent",
    instructions=instruction1,
    model="gpt-4o",
    tools=[analyze_image_url_with_flowus],
)
agent2 = Agent(
    name="权限说明文字判断Agent",
    instructions=instruction2,
    model="gpt-4o"
)
agent3 = Agent(
    name="Fridalog调用判断agent",
    instructions=instruction3,
    model="gpt-4o",
    tools=[filter_related_apis_from_fridalog]
)
agent4 = Agent(
    name="合规性报告生成Agent",
    instructions=instruction4,
    model="gpt-4o"
)

In [ ]:
#Agent执行函数
async def run_agent1(image_records: list[dict]):
    print("开始分析截图中的权限信息...")
    messages = [
        {
            "role": "system",
            "content": (
                "你是一个截图分析专家，用户会给你多张截图的图片 URL 和对应文件名。"
                "你必须逐张调用工具 analyze_image_url_with_flowus 来对每张图片进行识别分析，并根据返回结果完成以下任务：\n\n"
                "1. 提取图片中文字 raw_text，记录到 debug_output；\n"
                "2. 如果提取到了权限说明（permission_info 非空），记录到 agent2，并判断类型（type），记录到 agent3；\n"
                "3. 如果没有提取到权限说明，则 agent2.success 为 false，agent3.type 和 timestamp 为 null；\n"
                "4. 构建最终 JSON 输出，字段包含 from、status、agent2、agent3、debug_output；\n"
                "你必须逐张使用工具 analyze_image_url_with_flowus 获取每张图的识别内容。"
            )
        },
        {
            "role": "user",
            "content": "请分析以下截图中的权限信息，每张图都需要识别：\n" +
                       "\n".join(
                           [f"- 文件名：{record['filename']}\n  图像链接：{record['image_url']}" for record in image_records]
                       )
        }
    ]

    with trace("Agent1 权限识别流程"):
        result = await Runner.run(agent1, messages)

    print("✅ 分析完成")
    return result.final_output
async def run_agent2(agent2, agent1_output: dict) -> str:
    """
    输入完整的 agent1_output（包含 package 字段），将其传给 Agent2 并判断每条权限说明是否合规。
    """
    message = [
        {
            "role": "user",
            "content": json.dumps(agent1_output, ensure_ascii=False, indent=2)
        }
    ]

    with trace("Agent2 表述规范性分析"):
        result = await Runner.run(agent2, message)

    return result.final_output
async def run_agent3(agent3, agent1_output: dict, folder_path: str):
    """
    执行 Agent3，使用 agent1_output 和 frida.log 所在文件夹构造输入 message 并启动 Agent3。

    不再调用 filter_related_apis_from_fridalog 工具，也不使用 build_agent3_input。
    """
    with trace("Agent3 敏感API行为分析"):
        # 加载 frida 日志原始文本
        frida_log = load_frida_log(folder_path)

        if not frida_log.strip():
            print("❌ 未读取到有效的 Frida 日志")
            return {
                "from": "agent3",
                "package": agent1_output.get("package", "unknown"),
                "status": "true",
                "matched_apis": []
            }

        # 提取 type 和 description
        types = []
        for item in agent1_output.get("agent3", []):
            if item.get("success") and item.get("type"):
                types.append(item["type"])
        types = list(set(types))

        if not types:
            print("❌ Agent1 中无有效权限类型")
            return {
                "from": "agent3",
                "package": agent1_output.get("package", "unknown"),
                "status": "true",
                "matched_apis": []
            }

        # 构造输入 JSON 对象（Agent3 prompt 需要的结构）
        agent3_input = {
            "package": agent1_output.get("package", "unknown"),
            "types": types,
            "description": "\n".join([x.get("permission_info", "") for x in agent1_output.get("agent2", []) if x.get("permission_info")]),
            "backtrace": frida_log.strip()
        }

        # 构造对话消息
        messages = [
            {"role": "user", "content": json.dumps(agent3_input, ensure_ascii=False, indent=2)}
        ]

        # 执行 agent3
        result = await Runner.run(agent3, messages)
        return result.final_output
async def run_agent4(agent4, agent4_input: list) -> str:
    """
    执行 Agent4 合规性分析，输入为包含两个 JSON 的列表。

    参数:
        agent4: Agent4 实例
        agent4_input (list): 一个包含两个 JSON（agent1/2 和 agent3 输出）的列表

    返回:
        str: Agent4 输出的合规性分析报告
    """
    messages = [
        {"role": "user", "content": json.dumps(obj, ensure_ascii=False, indent=2)}
        for obj in agent4_input
    ]

    with trace("Agent4 合规性终端分析"):
        result = await Runner.run(agent4, messages)

    return result.final_output

In [80]:
#数据处理函数
def parse_agent_output(raw_str: Union[str, dict]) -> dict:
    """
    解析 Agent 输出，支持修复字符串中嵌套未转义的引号问题。
    """
    if isinstance(raw_str, dict):
        return raw_str

    raw = raw_str.strip()
    if raw.startswith("```json") and raw.endswith("```"):
        raw = raw[len("```json"):].strip()
        raw = raw[:-3].strip()
    elif raw.startswith("```") and raw.endswith("```"):
        raw = raw[3:-3].strip()
    return json.loads(raw)
def build_agent4_input(json1: dict, json2: dict) -> list:
    """
    构建 Agent4 的输入格式，将两个 JSON（分别为 agent1/agent2 和 agent3 的输出）原样组合为一个输入列表。

    参数:
        json1 (dict): 来自 agent1 或 agent2 的输出 JSON
        json2 (dict): 来自 agent3 的输出 JSON

    返回:
        list: 传入 Agent4 的输入消息格式
    """
    return [
        {"role": "user", "content": json.dumps(json1, ensure_ascii=False, indent=2)},
        {"role": "user", "content": json.dumps(json2, ensure_ascii=False, indent=2)}
    ]
def dispatch_agent4_input(agent1_output: dict, agent2_output: dict, agent3_output: dict) -> list:
    """
    根据 agent1_output 的 status 字段判断构建 agent4_input 所需的两个 JSON。
    - 若 agent1_output["status"] 为 False，说明权限说明不可见，使用 agent1 和 agent3；
    - 否则说明说明可见，使用 agent2 和 agent3。

    返回：
        list: 包含两个 JSON 的列表，用于输入 Agent4。
    """
    if not agent1_output.get("status", False):
        return [agent1_output, agent3_output]
    else:
        return [agent2_output, agent3_output]

In [59]:
#主执行函数
folder_path = r"D:\AgentProject\testAgent\mcp_service\testfridalogfolder\app.qmjita.cn"

package_name = os.path.basename(folder_path.rstrip("\\/"))
image_records = build_image_records(folder_path)
print(image_records)

✅ 上传成功: app.qmjita.cn_1755787770000.jpg
[{'filename': 'app.qmjita.cn_1755787770000.jpg', 'image_url': 'https://i.ibb.co/KxjZNYQ5/9d458ad362d5.jpg'}]


In [ ]:
#Agent1测试
agent1_output = await run_agent1(image_records)
# Agent1输出处理并新增package字段
agent1_output = parse_agent_output(agent1_output)
new_agent1_output = OrderedDict()
for k, v in agent1_output.items():
    new_agent1_output[k] = v
    if k == "from":
        new_agent1_output["package"] = package_name
agent1_output = new_agent1_output
print(agent1_output)

In [ ]:
#Agent2执行
agent2_input = agent1_output
agent2_output = await run_agent2(agent2, agent2_input)
print(agent2_output)

In [ ]:
#Agent3执行
agent3_output = await run_agent3(agent3, agent1_output, folder_path)

In [89]:
#Agent4执行
raw_agent4_input = dispatch_agent4_input(agent1_output, agent2_output, agent3_output)
agent4_input = build_agent4_input(*raw_agent4_input)
agent4_output = await run_agent4(agent4, agent4_input)
#报告可视化输出
display(Markdown(agent4_output))

# 移动应用隐私合规性评估报告

应用名称：**全民吉他**
包名：**app.qmjita.cn**

## 1. 权限说明概览

### 权限说明状态
应用的权限说明状态：**缺失**。  
此应用未能提供完整且可视的权限使用说明。这是符合隐私合规的严重问题，用户无法明确知晓应用请求的权限目的与使用范围。

### 权限类型与描述
- **管理电话权限** (`manage phone call`): 缺乏具体用途说明。

## 2. 表述规范性分析

由于应用缺乏有效的权限说明，我们暂时无法针对权限说明的明确性、可读性以及无诱导性进行充分的分析。然而，下述内容为此类情况提供了一些修订建议。

### 权限说明重写建议
- **管理电话权限**
  - 原始说明缺失。
  - 重写建议：`“电话权限用于接收来电提醒功能”`。

## 3. 用途一致性分析

### 动态分析结果
通过 Frida 动态分析，发现应用存在敏感 API 调用行为与截图时间戳匹配问题，这表明实际调用的某些敏感权限用途未在说明中明确披露。

**检测到的Sensitive API调用：**
1. `android.telephony.TelephonyManager.getDeviceId` 用于获取设备ID，存在未披露用途风险。
2. `android.telephony.TelephonyManager.getSubscriberId` 用于获取用户标识信息，需检查是否违反用户隐私政策。

### 相关SDK/模块
- Bytedance SDK
- Kwad SDK

上述 SDK 调用皆涉及设备标识及用户信息，需明确其必要性与合法用途。

## 4. 合规性评估与改进建议

### 合规性状况
该应用当前可能违反如下条款：
- [GDPR-13.1]: 应明确告知数据主体所收集个人数据的用途。
- [PIPL-23.1]: 应征得用户单独同意并明确告知其数据处理活动的范围和目的。

### 建议改进措施
1. **补充并公开权限说明**  
   - 明确披露各敏感权限的目的、使用场景及数据处理流程。
   - 示例：`“电话权限用于接收来电提醒功能”`。应避免抽象、常规或诱导性的说明。

2. **加强用户告知与同意流程**
   - 应在权限申请阶段提供详细的使用目的说明，并征得用户明确同意，确保与实际使用保持一致。
  
3. **优化权限调用策略**
   - 对于 `android.telephony.TelephonyManager.getDeviceId` 和 `android.telephony.TelephonyManager.getSubscriberId` 的调用需限制在合理需求范围内，并增加相应日志审查机制。

**总结：** 该应用需加快落实合规改进，补充及优化权限说明，增强用户告知与同意机制，确保其操作符合GDPR及PIPL的标准，从而降低合规风险。